In [ ]:
import sys
sys.path.append("/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/PSPNet-Keras-tensorflow") # access submodules
sys.path = [p for p in sys.path if not p.startswith('/local/scratch') or p.startswith('/local/scratch/js2173')]

In [ ]:
import os
from os.path import splitext, join
import numpy as np
from scipy import misc, ndimage
from keras import backend as K
from keras.models import model_from_json, load_model
import tensorflow as tf
import layers_builder as layers
from python_utils import utils
from python_utils.preprocessing import preprocess_img


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.id
sess = tf.Session()
K.set_session(sess)

